In [ ]:
from unet import UNet
import torch

unet_model_path = "unet_model.pth"

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
unet_model = UNet().to(device)
unet_model.load_state_dict(torch.load(unet_model_path))
unet_model.eval()


In [2]:
from dataset import RESNETDataset
from torch.utils.data import DataLoader

normal_dir = '../chest_xray/train/NORMAL'
pneumonia_dir = '../chest_xray/train/PNEUMONIA'

train_dataset = RESNETDataset(unet_model, device, normal_dir, pneumonia_dir, target_size=(256, 256))
train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True)

In [ ]:
from resnet import ResNetCNN
import torch.optim as optim
import torch.nn as nn

#device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = ResNetCNN().to(device)
criterion = nn.CrossEntropyLoss()
#optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.9)
optimizer = optim.RMSprop(model.parameters(), lr=1e-4)

# 모델 저장 경로
model_save_path = "resnet_model.pth"

In [ ]:
# 모델 학습
min = 999
num_epochs = 30
for epoch in range(num_epochs):
    model.train()
    epoch_loss = 0

    for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device)

        optimizer.zero_grad()
        outputs = model(images.repeat(1, 3, 1, 1))
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        epoch_loss += loss.item()

    print(f"Epoch {epoch + 1}/{num_epochs}, Loss: {epoch_loss:.5f}")
    if min > epoch_loss:
        torch.save(model.state_dict(), model_save_path)
        print(f"Model {epoch + 1} saved to {model_save_path}")
        min = epoch_loss


# 학습된 모델 저장
#torch.save(model.state_dict(), model_save_path)
#print(f"Model saved to {model_save_path}")
